In [ ]:
# This Helper fetches de resolver dataset from the EBI identifiers.org platform, to transform it and put it into
# the new MongoDB based data backend.
# Author: Manuel Bernal Llinares <mbdebian@gmail.com>
import time
import random
import pymongo
import requests
import threading
import matplotlib.pyplot as plt
import data.models.hq.registry as hqregistry

%matplotlib inline

In [ ]:
# Set up the environment
identifiersorg_url_resolver_data_source = 'https://identifiers.org/rest/collections/expand'
# TODO Maybe get this from environment variables later on when everything is working
mongodb_username = 'devusername'
mongodb_password = 'devpassword'
mongodb_database = 'devdbregistry'
mongodb_endpoint = 'mongodb://localhost:27017'
mongodb_authendpoint = "mongodb://{}:{}@localhost:27017".format(mongodb_username, mongodb_password)

In [ ]:
# Helpers
def make_rest_request_content_type_json(url):
    # TODO - Magic number here!!!
    n_attempts = 42
    response = None
    while n_attempts:
        n_attempts -= 1
        try:
            response = requests.get(url, headers={"Content-Type": "application/json"})
        except Exception as e:
            # Any possible exception counts towards the attempt counter
            # Random wait - TODO - Another magic number!!!
            time.sleep(random.randint(30))
            continue
        if response.ok:
            return response.json()
        # Random wait - TODO - Another magic number!!!
        time.sleep(random.randint(10))
    response.raise_for_status()

In [ ]:
# Initialise pseudo-random number generator
random.seed(time.time())

In [ ]:
# Preparing Database
dbclient = pymongo.MongoClient(mongodb_authendpoint)
print("---> DB Client instantiated")
dbregistry = dbclient(mongodb_database)
print("\tDatabases: {}".format())

In [ ]:
# Retrieve the Resolution Source dataset
resolution_dataset_source = make_rest_request_content_type_json(identifiersorg_url_resolver_data_source)

In [ ]:
# Process PID entries
for pid_entry in resolution_dataset_source:
    print("---> Processing Prefix '{}' - '{}'".format(pid_entry['id'], pid_entry['prefix']))
    # Translate PID Entry to Namespace model
    namespace = hqregistry.Namespace()
    namespace.prefix = pid_entry.get('prefix', None)
    namespace.mirId = pid_entry.get('id', None)
    namespace.name = pid_entry.get('name', None)
    namespace.pattern = pid_entry.get('pattern', None)
    namespace.description = pid_entry.get('definition', None)
    # TODO creation and modification dates are missing from the dataset
    # There is no deprecation date for now
    # Flag all namespaces as 'NOT Deprecated' for now
    namespace.deprecated = False